In [2]:
import pandas as pd

In [3]:
hotel = pd.read_csv('bookings.csv', encoding='utf-8', sep=';')

In [4]:
hotel.shape

(119390, 21)

In [5]:
hotel.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [6]:
hotel.head(7)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [10]:
def rename_file(name):
    return name.replace(' ', '_').lower()

In [33]:
hotel = hotel.rename(columns=rename_file)

Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.

In [96]:
hotel.query('is_canceled == 0') \
    .country \
    .value_counts()[:5]

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64


На сколько ночей в среднем бронируют отели разных типов?

In [98]:
hotel.groupby('hotel', as_index=False) \
    .agg({'stays_total_nights': 'mean'}) \
    .round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?

In [100]:
hotel.query('reserved_room_type != assigned_room_type').shape[0]

14917


Проанализируйте даты запланированного прибытия. 
– На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?
– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов

In [104]:
hotel.query('arrival_date_year == 2016') \
    .arrival_date_month \
    .value_counts()

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

In [105]:
hotel.query('arrival_date_year == 2017') \
    .arrival_date_month \
    .value_counts()

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

In [111]:
hotel.query('hotel == "City Hotel"') \
    .groupby('arrival_date_year', as_index=False) \
    .agg({'is_canceled' : 'sum'})

,arrival_date_year,is_canceled
0,2015,6004
1,2016,15407
2,2017,11691


In [114]:
(
    hotel
        .query('hotel == "City Hotel" and is_canceled == 1')
        .groupby('arrival_date_year')
        .arrival_date_month
        .value_counts()
)

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [127]:
hotel[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

Создайте колонку total_kids, объединив children и babies. Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?

In [131]:
hotel['total_kids'] = hotel['children'] + hotel['babies']

In [141]:
hotel.groupby('hotel') \
    .agg({'total_kids':'mean'}) \
    .round(2) \
    .max()

total_kids    0.14
dtype: float64

Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.

In [143]:
hotel['has_kids'] = hotel.total_kids > 0

In [154]:
no_kids = hotel.query('has_kids == False and is_canceled == 1').shape[0] / hotel.query('has_kids == False').shape[0]
no_kids = round(no_kids * 100, 2)

In [158]:
yes_kids = hotel.query('has_kids == True and is_canceled == 1').shape[0] / hotel.query('has_kids == True').shape[0]
yes_kids = round(yes_kids * 100, 2)

In [159]:
no_kids > yes_kids

True